In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from model_AlexNet import CustomAlexNet

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_set = torchvision.datasets.CIFAR10(root='../cifar10', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True)

test_set = torchvision.datasets.CIFAR10(root='../cifar10', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False)

100.0%


Extracting ../cifar10\cifar-10-python.tar.gz to ../cifar10
Files already downloaded and verified


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

cuda:0 is available


In [5]:
model = CustomAlexNet().to(device)
print(model)

CustomAlexNet(
  (conv1): Sequential(
    (0): Conv2d(3, 48, kernel_size=(4, 4), stride=(2, 2))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(48, 100, kernel_size=(2, 2), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(100, 150, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(150, 150, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv5): Sequential(
    (0): Conv2d(150, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (fclayer): Sequential(
    (0): 

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [7]:
from tqdm import tqdm

loss_ = []
batch_num = len(train_loader)

for epoch in range(10):
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}", leave=False)

    for i, data in enumerate(progress_bar):
        input, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(input)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    loss_.append(running_loss / batch_num)
    print('[%d] loss: %.3f' %(epoch + 1, running_loss / len(train_loader)))


Epoch 1:   0%|          | 0/6250 [00:00<?, ?it/s]

[1] loss: 1.813


[2] loss: 1.534


[3] loss: 1.430


[4] loss: 1.371


...

[6] loss: 1.284


[7] loss: 1.274


[8] loss: 1.250


[9] loss: 1.249


[10] loss: 1.236


In [8]:
Model_PATH = './Alex.pt'
torch.save(model.state_dict(), Model_PATH)

In [9]:
model = CustomAlexNet().to(device)
model.load_state_dict(torch.load(Model_PATH))

<All keys matched successfully>

In [10]:
correct = 0
total = 0

with torch.no_grad(): 
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum().item() 

print(f'accuracy of 10000 test images: {100*correct/total}%')

accuracy of 10000 test images: 56.97%
